## Loading packages

In [ ]:
# Import required packages
import warnings
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Suppress Cartopy warnings (safe to keep even if Cartzopy is not used)
warnings.filterwarnings("ignore")

## Function definition

In [ ]:
def fix_time_variable(ds):
    """
    Description: fix the time of cam file to have the time at
       the middle of the interval instead of the end the interval.
    Argument:
       ds: dataset
    Return:
       ds: dataset with the time variable at the middle of interval.
    """
    time = ds["time"]
    time = xr.DataArray(
        ds["time_bnds"].load().mean(dim="nbnd").values, dims=time.dims, attrs=time.attrs
    )
    ds["time"] = time
    ds.assign_coords(time=time)
    return ds


def ann_mean(ds):
    """
    Description: compute the annual mean of a monthly mean dataset
    Argument:
       ds: dataset "
    Return:
       ds_ann: annual mean
    """
    ds_gb = ds.groupby("time.year")
    ds_ann = ds_gb.mean()
    return ds_ann


def global_mean(variable):
    """
    Description: compute the global mean (weigthed) of a variable
    Argument:
       ds: variable
    Return:
       ds_ann: annual mean
    """
    weights = np.cos(np.deg2rad(variable.lat))
    variable_weigthed = variable.weighted(weights)
    variable_mean = variable_weigthed.mean(("lon", "lat"))
    return variable_mean


def compute_var_g_ann(case, var):
    """
    Description: compute the annual global mean (weigthed) of
    the variable "var".
    Argument:
       case = casename
       var = variable name
    """
    filename = (
        "/glade/campaign/cgd/amp/amwg/ts/" + case + "/" + case + "." + var + ".nc"
    )
    print({filename})
    ds = xr.open_dataset(filename)

    # ds = fix_time_variable(ds)
    ds.mean(["lat", "time"])
    var_ann = ann_mean(ds[var])
    var_g_ann = global_mean(var_ann)
    return var_g_ann


def lat_lon_mean(variable, lat1, lat2, lon1, lon2):
    """
    Description: Compute the mean (weighted by cosine of latitude) of a variable within a specified latitude (lat1-lat2) and longitude (lon1-lon2) range.

    Arguments:
        variable: xarray DataArray representing the variable to be averaged.
        lat1: Starting latitude for the range.
        lat2: Ending latitude for the range.
        lon1: Starting longitude for the range.
        lon2: Ending longitude for the range.

    Return:
        variable_mean: The mean value of the variable within the specified lat/lon range.
    """
    # Select the subset of data within the specified latitude and longitude ranges
    variable_subset = variable.sel(lat=slice(lat1, lat2), lon=slice(lon1, lon2))

    # Compute weights as the cosine of the latitude, adjusted for the selected range
    weights = np.cos(np.deg2rad(variable_subset.lat))

    # Apply weighted mean over the subset
    variable_weighted = variable_subset.weighted(weights)
    variable_mean = variable_weighted.mean(("lon", "lat"))

    return variable_mean


def lat_lon_mean_norm(variable, lat1, lat2, lon1, lon2):
    """
    Description: Compute the mean (weighted by cosine of latitude) of a variable within a specified latitude (lat1-lat2) and longitude (lon1-lon2) range.

    Arguments:
        variable: xarray DataArray representing the variable to be averaged.
        lat1: Starting latitude for the range.
        lat2: Ending latitude for the range.
        lon1: Starting longitude for the range.
        lon2: Ending longitude for the range.

    Return:
        variable_mean: The mean value of the variable within the specified lat/lon range.
    """
    # Select the data subset within the specified latitude and longitude ranges
    variable_subset = variable.sel(lat=slice(lat1, lat2), lon=slice(lon1, lon2))

    # Compute weights as the cosine of the latitude, adjusted for the selected range
    # Ensure weights are normalized (sum to 1) over the selected latitude range for accurate weighting
    latitudes = variable_subset.lat
    weights = np.cos(np.deg2rad(latitudes))
    weights /= weights.sum(dim="lat")

    # Apply weighted mean over the subset
    variable_weigthed = variable_subset.weighted(weights)
    variable_mean = variable_weigthed.mean(("lon", "lat"))

    return variable_mean


def compute_var_lat_lon_ann(case, var, lat1, lat2, lon1, lon2):
    """
    Description: Compute the annual mean within a specified latitude (lat1-lat2) and longitude (lon1-lon2) range of the variable "var".

    Arguments:
       case = casename
       var = variable name
       lat1 = starting latitude
       lat2 = ending latitude
       lon1 = starting longitude
       lon2 = ending longitude
    """
    filename = (
        "/glade/derecho/scratch/hannay/timeseries/"
        + case
        + "/"
        + case
        + "."
        + var
        + ".nc"
    )
    filename = (
        "/glade/campaign/cgd/amp/amwg/ts/" + case + "/" + case + "." + var + ".nc"
    )
    ds = xr.open_dataset(filename)
    # ds = fix_time_variable(ds)

    # Select the subset of data within the specified latitude and longitude ranges before calculating the annual mean
    ds_subset = ds.sel(lat=slice(lat1, lat2), lon=slice(lon1, lon2))

    var_ann = ann_mean(ds_subset[var])
    var_lat_lon_ann = lat_lon_mean(var_ann, lat1, lat2, lon1, lon2)

    return var_lat_lon_ann


def compute_var_zonal_ann(case, var):
    """
    Description: compute the annual global mean (weigthed) of
    a variable var
    Argument:
       case = casename
       var = variable name
    """
    filename = (
        "/glade/campaign/cgd/amp/amwg/ts/" + case + "/" + case + "." + var + ".nc"
    )
    ds = xr.open_dataset(filename)
    # ds = fix_time_variable(ds)
    ds.mean(["lon"])
    var_zonal_ann = ann_mean(ds[var]).mean(["lon"])
    return var_zonal_ann


def compute_dTS(case1, case2):
    filename1 = (
        "/glade/derecho/scratch/hannay/timeseries/" + case1 + "/" + case1 + ".TS.nc"
    )
    filename2 = (
        "/glade/derecho/scratch/hannay/timeseries/" + case2 + "/" + case2 + ".TS.nc"
    )
    ds1 = xr.open_dataset(filename1)
    # ds1 = fix_time_variable(ds1)
    ds1 = ann_mean(ds1)
    ds2 = xr.open_dataset(filename2)
    # ds2 = fix_time_variable(ds2)
    ds2 = ann_mean(ds2)
    dTS = ds2.TS - ds1.TS.mean("year")
    return dTS

___ 
## Read Data

### Read CESM2-LE data

In [ ]:
filename = "/glade/work/hannay/CESM2-LE/TS_am_LENS2_first50.nc"
ds = xr.open_dataset(filename)

# Select years 1 to 165
TS_LE_hist = ds.TS.isel(year=slice(0, 165))

### Read weight for f09 grid

In [ ]:
filename = "/glade/derecho/scratch/hannay/timeseries/w.nc"
print({filename})
ds = xr.open_dataset(filename)
w = ds.w

### Read CESM output

In [ ]:
# Define cases with short labels

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "baseline",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 0,
        "descr": "baseline",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.194": {
        "label": "194",
        "offset": 0,
        "descr": "IC from G case",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.195": {
        "label": "195",
        "offset": 0,
        "descr": "KPP, etc",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.196": {
        "label": "196",
        "offset": 0,
        "descr": "Labsea submesoscale mixing",
    },
    "b.e30_beta06_megan.B1850C_MTt4s.ne30.004": {
        "label": "BMT T4s",
        "offset": 0,
        "descr": "Full chemistry",
    },
}


cases = {
    "b.e30_beta06_megan.B1850C_MTt4s.ne30.004": {
        "label": "BMT T4s",
        "offset": 0,
        "descr": "Full chemistry",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "baseline",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 93 - 1850,
        "descr": "baseline",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "baseline",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.194": {
        "label": "194",
        "offset": 0,
        "descr": "IC from G case",
    },
    "b.e30_beta06_megan.B1850C_MTt4s.ne30.004": {
        "label": "BMT T4s",
        "offset": 0,
        "descr": "Full chemistry",
    },
}

cases = {
    "b.e30_beta06_megan.B1850C_MTt4s.ne30.004": {
        "label": "BMT T4s",
        "offset": 0,
        "descr": "Full chemistry",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "baseline",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.194": {
        "label": "194",
        "offset": 0,
        "descr": "IC from G case",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.195": {
        "label": "195",
        "offset": 0,
        "descr": "KPP, etc",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.196": {
        "label": "196",
        "offset": 0,
        "descr": "Labsea submesoscale mixing",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.194": {
        "label": "194",
        "offset": 0,
        "descr": "IC from G case",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.195": {
        "label": "195",
        "offset": 0,
        "descr": "ocn package",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.196": {
        "label": "196",
        "offset": 0,
        "descr": "Labsea submesoscale mixing",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.197": {
        "label": "197",
        "offset": 0,
        "descr": "195 + old land files",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "197 + warmer/saltier",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.199": {
        "label": "199",
        "offset": 0,
        "descr": "explicit diffusion off",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 0,
        "descr": "latest CMIP7",
    },
    "b.e30_alpha07b.B1850C_LTso.ne30_t232_wgx3.201": {
        "label": "201",
        "offset": 0,
        "descr": "same as 194 with alpha07b + old land",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.194": {
        "label": "194",
        "offset": 0,
        "descr": "IC from G case",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.199": {
        "label": "199",
        "offset": 0,
        "descr": "explicit diffusion off",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.194": {
        "label": "194",
        "offset": 0,
        "descr": "IC from G case",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.195": {
        "label": "195",
        "offset": 0,
        "descr": "ocn package",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.197": {
        "label": "197",
        "offset": 0,
        "descr": "195 + old land files",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "197 + warmer/saltier",
    },
    "b.e30_alpha07b.B1850C_LTso.ne30_t232_wgx3.201": {
        "label": "201",
        "offset": 0,
        "descr": "same as 194 with alpha07b + old land",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "1850 with CMIP7 v1",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 93 - 1850,
        "descr": "20thC with CMIP7 v2",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.199": {
        "label": "199",
        "offset": 0,
        "descr": "1850 with CMIP7 v2",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "1850 with CMIP7 v1",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 93 - 1850,
        "descr": "20thC with CMIP7 v2",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.194": {
        "label": "194",
        "offset": 0,
        "descr": "IC from G case",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.195": {
        "label": "195",
        "offset": 0,
        "descr": "ocn package",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.197": {
        "label": "197",
        "offset": 0,
        "descr": "195 + old land files",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "197 + warmer/saltier",
    },
    "b.e30_alpha07b.B1850C_LTso.ne30_t232_wgx3.201": {
        "label": "201",
        "offset": 0,
        "descr": "same as 194 with alpha07b + old land",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "1850 with CMIP7 v1",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 93 - 1850,
        "descr": "20thC with CMIP7 v2",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 93,
        "descr": "1850 with CMIP7 v2",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "1850 with CMIP7 v1",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 93 - 1850,
        "descr": "20thC with CMIP7 v2",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 93,
        "descr": "1850 with CMIP7 v2",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 93 + 45,
        "descr": "20th with CMIP7 v2",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3_fixed_emission.200": {
        "label": "200",
        "offset": 93 + 45,
        "descr": "20th with 1850 CMIP7 v2",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "1850 with CMIP7 v1",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 93 - 1850,
        "descr": "20thC with CMIP7 v2",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 93,
        "descr": "1850 with CMIP7 v2",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "1850 with CMIP7 v1",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 93 - 1850,
        "descr": "20thC with CMIP7 v2",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 93,
        "descr": "1850 with CMIP7 v2",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.200": {
        "label": "200_hist",
        "offset": 93 + 45 - 1850,
        "descr": "20th with CMIP7 v2",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3_fixed_emission.200": {
        "label": "200_hist_pia",
        "offset": 93 + 45 - 1850,
        "descr": "20th with 1850 CMIP7 v2",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.199": {
        "label": "199",
        "offset": 0,
        "descr": "explicit diff off",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "197 + warmer/saltier",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.202": {
        "label": "202",
        "offset": 0,
        "descr": "198+199",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.209": {
        "label": "209",
        "offset": 0,
        "descr": "202+???",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.210": {
        "label": "210",
        "offset": 0,
        "descr": "202+???",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "1850 with CMIP7 v1",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.192": {
        "label": "192_hist",
        "offset": 93 - 1850,
        "descr": "20thC with CMIP7 v2",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 93,
        "descr": "1850 with CMIP7 v2",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.199": {
        "label": "199",
        "offset": 0,
        "descr": "explicit diff off",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "new ocn + warmer/saltier IC",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.202": {
        "label": "202",
        "offset": 0,
        "descr": "198+199",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.203": {
        "label": "203",
        "offset": 0,
        "descr": "202+dcs=500",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.204": {
        "label": "204",
        "offset": 0,
        "descr": "202+dcs=700",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.205": {
        "label": "205",
        "offset": 0,
        "descr": "202+clubb_detliq_rad=20",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.206": {
        "label": "206",
        "offset": 0,
        "descr": "202+num_cin=1",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.211": {
        "label": "211",
        "offset": 0,
        "descr": "202+clubb_detliq_rad=10",
    },
}

cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.202": {
        "label": "202",
        "offset": 0,
        "descr": "198+199",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.209": {
        "label": "209",
        "offset": 0,
        "descr": "202+???",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.210": {
        "label": "210",
        "offset": 0,
        "descr": "202+???",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 0,
        "descr": "1850 with CMIP7 v2",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.200": {
        "label": "200_hist",
        "offset": 45 - 1850,
        "descr": "20th with CMIP7 v2",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3_fixed_emission.200": {
        "label": "200_hist_pia",
        "offset": 45 - 1850,
        "descr": "20th with 1850 CMIP7 v2",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.192": {
        "label": "192",
        "offset": 0,
        "descr": "1850 with CMIP7 v1",
    },
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 93,
        "descr": "1850 with CMIP7 v2",
    },
}

cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.200": {
        "label": "200",
        "offset": 0,
        "descr": "1850 with CMIP7 v2",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3.200": {
        "label": "200_hist",
        "offset": 45 - 1850,
        "descr": "20th with CMIP7 v2",
    },
    "b.e30_beta06.BHISTC_LTso.ne30_t232_wgx3_fixed_emission.200": {
        "label": "200_hist_pia",
        "offset": 45 - 1850,
        "descr": "20th with 1850 CMIP7 v2",
    },
}


cases = {
    "b.e30_beta06.B1850C_LTso.ne30_t232_wgx3.194": {
        "label": "194",
        "offset": 0,
        "descr": "previous baseline: IC from G case",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + old land",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.207": {
        "label": "207",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + new land",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.202": {
        "label": "202",
        "offset": 0,
        "descr": "explicitdiffF",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.212": {
        "label": "212",
        "offset": 0,
        "descr": "scale_fac = 0.05",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.209": {
        "label": "209",
        "offset": 0,
        "descr": "scale_fac = 0.25",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.210": {
        "label": "210",
        "offset": 0,
        "descr": "scale_fac = 0.5",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.213": {
        "label": "213",
        "offset": 0,
        "descr": "scale_fac = 0.75",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "explicitdiffT",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.214": {
        "label": "214",
        "offset": 0,
        "descr": "CMIP7 emission final",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.215": {
        "label": "215",
        "offset": 0,
        "descr": "turn-off lw downscaling",
    },
}

cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + old land",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.207": {
        "label": "207",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + new land",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.216": {
        "label": "216",
        "offset": 0,
        "descr": "Updated ocean ",
    },
}

cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "explicit diffusion on",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.202": {
        "label": "202",
        "offset": 0,
        "descr": "explicit diffusion oofff",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.213": {
        "label": "213",
        "offset": 0,
        "descr": "partial explicit diffusion ",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + old land",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.207": {
        "label": "207",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + new land",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.216": {
        "label": "216",
        "offset": 0,
        "descr": "Updated ocean ",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.217": {
        "label": "217",
        "offset": 0,
        "descr": "Updated ocean ",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.218": {
        "label": "218",
        "offset": 0,
        "descr": "Updated ocean ",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.219": {
        "label": "219",
        "offset": 0,
        "descr": "Updated ocean ",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "explicit diffusion on",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.213": {
        "label": "213",
        "offset": 0,
        "descr": "partial explicit diffusion ",
    },
}


cases = {
    "b.e30_alpha07b_dev.BHISTC_LTso.ne30_t232_wgx3.214": {
        "label": "214_hist",
        "offset": 0,
        "descr": "20th C: CMIP7 emission final",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + old land",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.207": {
        "label": "207",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + new land",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.220": {
        "label": "220",
        "offset": 0,
        "descr": "Updated ocean ",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.222": {
        "label": "222",
        "offset": 0,
        "descr": "Updated ocean ",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + old land",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.224": {
        "label": "224",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + new land",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.225": {
        "label": "225",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + new land",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "pi control:CMIP7 emission v1",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.223": {
        "label": "223",
        "offset": 0,
        "descr": "pi control: CMIP7 emission final",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + old land",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.224": {
        "label": "224",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + new land",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.225": {
        "label": "225",
        "offset": 0,
        "descr": "new ocean + warm/salty IC + new land",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "pi control:CMIP7 emission v1",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.214": {
        "label": "214",
        "offset": 125,
        "descr": "pi control: CMIP7 emission final",
    },
    "b.e30_alpha07b_dev.BHISTC_LTso.ne30_t232_wgx3.214": {
        "label": "214_hist",
        "offset": 125 + 21 - 1850,
        "descr": "20th C: CMIP7 emission final",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "baseline (old land)",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.207": {
        "label": "207",
        "offset": 0,
        "descr": "198 + other land IC",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.228": {
        "label": "228",
        "offset": 0,
        "descr": "reverting all ocean changes",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "pi control:CMIP7 emission v1",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.223": {
        "label": "223",
        "offset": 0,
        "descr": "pi control: CMIP7 emission final",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "pi control:CMIP7 emission v1",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.214": {
        "label": "214",
        "offset": 125,
        "descr": "pi control: CMIP7 emission final",
    },
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.215": {
        "label": "215",
        "offset": 125,
        "descr": "20th C: CMIP7 emission final",
    },
}


cases = {
    "b.e30_alpha07b_dev.B1850C_LTso.ne30_t232_wgx3.198": {
        "label": "198",
        "offset": 0,
        "descr": "baseline (old land)",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.228": {
        "label": "228",
        "offset": 0,
        "descr": "disabling wave attenuation",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.231": {
        "label": "231",
        "offset": 0,
        "descr": "",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.232": {
        "label": "232",
        "offset": 0,
        "descr": "",
    },
}

In [ ]:
# Define cases with short labels

cases = {
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.234": {
        "label": "234",
        "offset": 0,
        "descr": "baseline",
    },
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.247": {
        "label": "247",
        "offset": 0,
        "descr": "FSD",
    },
}

In [ ]:
# Compute global annual mean TS for each case
TS_global = {}
for casename, info in cases.items():
    label = info["label"]
    print(label)
    print(casename)
    TS_global[label] = compute_var_g_ann(casename, "TS")

In [ ]:
# read ICEFRAC for Lab sea
lat1 = 53.5
lat2 = 65.3
lon1 = 360 - 63.5
lon2 = 360 - 47.5

# Compute global annual mean TS for each case
ICE_labsea = {}
for casename, info in cases.items():
    label = info["label"]
    ICE_labsea[label] = compute_var_lat_lon_ann(
        casename, "ICEFRAC", lat1, lat2, lon1, lon2
    )

In [ ]:
# Compute global annual mean RESTOM for each case
RESTOM_global = {}
RESTOM_global_after20 = {}  # dropping the first 20 years of the run
restom_5yr_avg = {}
for casename, info in cases.items():
    label = info["label"]

    # --- restom global ---
    RESTOM_global[label] = compute_var_g_ann(casename, "RESTOM")

    # --- drop first 20 years ---
    first_year = RESTOM_global[label].year.min().item()
    restom_after20 = RESTOM_global[label].where(
        RESTOM_global[label].year >= first_year + 20, drop=True
    )
    RESTOM_global_after20[label] = restom_after20

    # 5-year running mean
    restom_5yr_avg[label] = RESTOM_global[label].rolling(year=5, center=True).mean()

# 5 year running averaging but RESTOM in  legend is not running average

In [ ]:
import matplotlib.pyplot as plt
import itertools

# Set up plot
fig = plt.figure(figsize=(20, 10))
plt.subplots_adjust(hspace=0.3)
font_size_axis = 12

# Define automatic styles
colors = ["black", "red", "blue", "green", "orange"]
colors = ["black", "red", "blue", "green"]
colors = ["black", "red", "blue"]
linestyles = ["-", "--", ":", "-."]

# First subplot
ax = fig.add_subplot(2, 2, 1)
style_cycle = itertools.cycle(
    [(color, ls) for ls in linestyles for color in colors]
)  # cycle through colors, then linestyle
for casename, info in cases.items():
    label = info["label"]
    offset = info["offset"]
    ts = TS_global.get(label)
    if ts is not None:
        color, linestyle = next(style_cycle)
        ax.plot(
            ts.year + offset,
            ts,
            color=color,
            linestyle=linestyle,
            label=label,
            linewidth=2.0,
        )
ax.set_xlabel("Time (year)", fontsize=font_size_axis)
ax.set_ylabel("TS (K)", fontsize=font_size_axis)
ax.set_title("TS", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=14)
ax.grid()

# Second subplot
ax = fig.add_subplot(2, 2, 2)
style_cycle = itertools.cycle([(color, ls) for ls in linestyles for color in colors])
for casename, info in cases.items():
    label = info["label"]
    offset = info["offset"]
    ice_labsea = ICE_labsea.get(label)
    if ice_labsea is not None:
        color, linestyle = next(style_cycle)
        ax.plot(
            ice_labsea.year + offset,
            ice_labsea,
            color=color,
            linestyle=linestyle,
            label=label,
            linewidth=2.0,
        )

ax.set_xlabel("Time (year)", fontsize=font_size_axis)
ax.set_ylabel("ICEFRAC", fontsize=font_size_axis)
ax.set_title("ICEFRAC (Lab sea)", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=14)
ax.set_ylim(0, 0.5)
ax.grid()

# Third subplot
ax = fig.add_subplot(2, 2, 3)

style_cycle = itertools.cycle([(color, ls) for ls in linestyles for color in colors])
for casename, info in cases.items():
    label = info["label"]
    offset = info["offset"]
    descr = info["descr"]
    restom_5yr = restom_5yr_avg.get(label)
    restom = RESTOM_global.get(label)
    if restom_5yr is not None:
        color, linestyle = next(style_cycle)
        ax.plot(
            restom_5yr.year + offset,
            restom_5yr,
            color=color,
            linestyle=linestyle,
            label=f"{label}: {restom.mean().item():.2f} W/m2  ({descr})",
            linewidth=2.0,
        )

ax.set_xlabel("Time (year)", fontsize=font_size_axis)
ax.set_ylabel("RESTOM (W/m2)", fontsize=font_size_axis)
ax.set_title("RESTOM (5-year running avg)", fontsize=20)
ax.tick_params(axis="both", which="major", labelsize=14)

# Adding legend with title
legend = ax.legend(
    loc="center left", bbox_to_anchor=(1.3, 0.5), fontsize="16", title="RESTOM"
)
legend.get_title().set_fontsize("18")  # Setting the font size of the legend title

ax.grid()

plt.show()